In [14]:
import openrouteservice as ors
import folium

#client = ors.Client(key='5b3ce3597851110001cf62480413a152b510459cabf5e54bcfb4a7a9')
#client = ors.Client(key='5b3ce3597851110001cf6248f4b857eea0554d93886b4c09e748b2bf')
client = ors.Client(key='5b3ce3597851110001cf6248e0bab62fe9d04167aed0c5bbd383e116')

In [15]:
import re
import pandas as pd
import math

import numpy as np

from operator import itemgetter
from heapq import nsmallest

from itertools import chain

import time

In [7]:
df_property = pd.read_csv('../data/curated/property.csv')

property_coor = []
for index, row in df_property.iterrows():
    lat = float(re.findall(r'\[(-\d+.\d+)',str(row['coordinates']))[0])
    long = float(re.findall(r'(\d+.\d+)\]',str(row['coordinates']))[0])
    property_coor.append((long,lat))

print(len(property_coor))

df_train = pd.read_csv('../data/curated/train_coord.csv')

train_coor = []
for index, row in df_train.iterrows():
    lat = row['1']
    long = row['0']
    train_coor.append((long,lat))
    
print(len(train_coor))

15222
332


In [16]:
import geopy.distance

In [11]:
# find the closest 3 train stations for each property based on distance
# api has accessed limits so that it is difficult to track the route between each train station and each property


def get_train(index):

    # calculate distance between the property and each train station
    distances = [geopy.distance.geodesic((property_coor[index][1], property_coor[index][0]), (train_coor[j][1], train_coor[j][0])).km for j in range(len(train_coor))]

    # find the 3 nearest train stations
    index = list(range(len(train_coor)))
    pairs = zip(index, distances)
    result = nsmallest(3, pairs, key=itemgetter(1))
    index_list = [i[0] for i in result]

    # store the coordinates of the 3 nearest train stations
    coor_list = [train_coor[i] for i in index_list]

    return coor_list

# for each property, get corresponding closest train stations
coor_list = []

for i in range(len(property_coor)):
    result = get_train(i)
    coor_list.append(result)

print(len(coor_list))

15222


In [17]:
def get_train_ors(iter):


    flatten_list = list(chain.from_iterable(coor_list[iter*34:iter*34+34]))
    combine_coor = property_coor[iter*34:iter*34+34] + flatten_list

    if iter == len(df_property)//34:
        count = len(df_property)-iter*34
    else:
        count = 34

    matrix = client.distance_matrix(
        locations=combine_coor,
        profile='driving-car',
        metrics=['distance', 'duration'],
        validate=False,
        destinations=list(range(count,len(combine_coor))),
        sources=list(range(count)),
    )

    duration_list = []
    distance_list = []
    
    train_coor_list = []


    for i in range(count):

        min_index = np.argmin(matrix['durations'][i][i*3:i*3+3])
        min_index_flat = min_index+i*3
        duration_list.append(matrix['durations'][i][min_index_flat])
        distance_list.append(matrix['distances'][i][min_index_flat])

        train_coor_list.append(coor_list[iter*34+i][min_index])


    #print("Durations in secs: {}\n".format(matrix['durations']))
    #print("Distances in m: {}".format(matrix['distances']))

    # we only want to call the whole function once at each main iteration
    result_list = []
    result_list.append(duration_list)
    result_list.append(distance_list)
    result_list.append(train_coor_list)

    return result_list

# A total of 3500 routes is allowed for each call of api, so that we can pass 34 groups of property 
# with corresponding 3 closest schools at each time

duration_list = []
distance_list = []
train_coor_list = []

num_call = len(df_property)//34 + 1
for i in range(num_call):
    result = get_train_ors(i)
    
    if i%40 == 0:
        time.sleep(60)
    
    duration_list.append(result[0])
    distance_list.append(result[1])
    train_coor_list.append(result[2])

print(duration_list)
print(distance_list)
print(train_coor_list)

/Users/a123/opt/anaconda3/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/a123/opt/anaconda3/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/a123/opt/anaconda3/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/a123/opt/anaconda3/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/a123/opt/anaconda3/lib/py

[[249.2, 137.31, 207.3, 88.92, 249.26, 207.39, 161.81, 256.65, 135.62, 122.61, 236.28, 171.35, 92.27, 111.09, 209.88, 133.1, 189.06, 252.78, 98.36, 173.8, 202.17, 202.17, 203.03, 87.43, 194.05, 131.41, 94.34, 98.61, 238.48, 246.8, 161.81, 124.47, 214.07, 240.09], [238.48, 261.45, 48.9, 238.48, 246.8, 240.09, 240.09, 250.58, 240.09, 62.92, 238.48, 262.23, 205.07, 84.07, 62.92, 205.07, 197.19, 216.92, 82.14, 254.44, 238.48, 124.47, 124.47, 124.47, 261.2, 238.48, 50.37, 172.43, 238.48, 124.47, 238.48, 227.91, 205.07, 186.12], [90.54, 252.78, 226.87, 238.48, 197.19, 238.48, 240.09, 238.48, 269.69, 238.48, 240.09, 259.37, 216.92, 124.47, 230.04, 48.9, 48.9, 240.09, 147.22, 248.69, 130.35, 269.69, 257.63, 238.48, 147.79, 259.96, 254.44, 147.79, 254.97, 182.64, 182.64, 69.26, 229.16, 226.2], [270.98, 238.48, 87.43, 197.19, 216.92, 242.11, 259.96, 269.69, 116.65, 259.96, 269.69, 216.92, 226.87, 242.11, 202.17, 141.73, 147.79, 124.47, 87.43, 50.37, 249.16, 252.78, 182.64, 124.47, 246.8, 261.2, 

In [18]:
flatten_duration_list = list(chain.from_iterable(duration_list))
flatten_distance_list = list(chain.from_iterable(distance_list))
flatten_train_coor_list = list(chain.from_iterable(train_coor_list))

print(len(flatten_distance_list))

15222


In [19]:
df = pd.read_csv('../data/curated/property.csv')

df['driving_time_to_station'] = flatten_duration_list
df['distance_to_station'] = flatten_distance_list
df['closest_station_coor'] = flatten_train_coor_list

df.to_csv('../data/curated/property_train.csv')